# Performs double-counting removal

In [1]:
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
mapping_technologies = pd.read_csv('energyscope_data/mapping_linked.csv') # mapping between the esm technologies and LCI datasets
mapping_esm_flows_to_CPC = pd.read_csv('energyscope_data/mapping_esm_flows_to_CPC.csv') # mapping between the esm flows and CPC categories
model = pd.read_csv('energyscope_data/model.csv') # input output definitions of the esm technologies 
technology_compositions = pd.read_csv('energyscope_data/technology_compositions.csv') # compositions of the esm technologies
technology_specifics = pd.read_csv('energyscope_data/technology_specifics.csv') # specific requirements for the esm technologies

In [3]:
bd.projects.set_current("ei3.8-mescal")

In [4]:
db = concatenate_databases(list(mapping_technologies.Database.unique()))

In [5]:
new_db_name = 'energyscope_QC_2020'

## New database without foreground regionalization

### Database creation

In [6]:
create_esm_database(
    mapping=mapping_technologies, 
    model=model, 
    tech_specifics=technology_specifics, 
    technology_compositions=technology_compositions, 
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC, 
    main_database=db,
    esm_db_name=new_db_name,
    results_path_file='energyscope_data/results/'
)

AEC_OG
ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_BIOMETHANE
BIOGAS_SMR
BIOGAS_SMR_CCS
BIOMASS_ETHANOL
BIOMASS_GAS_EF_H2
BIOMASS_GAS_EF_H2_CCS
BIOMASS_GAS_FB_H2
BIOMASS_GAS_FB_H2_CCS
BUS_BIODIESEL_B100_SD
BUS_BIODIESEL_B20_SD
BUS_CNG_SD
BUS_DIESEL_SD
BUS_EV_SD
BUS_FC_CH4_SD
BUS_FC_H2_SD
BUS_HY_DIESEL_SD
BUS_PROPANE_SD
CAR_BIODIESEL_B100_ELD
CAR_BIODIESEL_B100_LD
CAR_BIODIESEL_B100_MD
CAR_BIODIESEL_B100_SD
CAR_BIODIESEL_B20_ELD
CAR_BIODIESEL_B20_LD
CAR_BIODIESEL_B20_MD
CAR_BIODIESEL_B20_SD
CAR_CNG_ELD
CAR_CNG_LD
CAR_CNG_MD
CAR_CNG_SD
CAR_DIESEL_ELD
CAR_DIESEL_LD
CAR_DIESEL_MD
CAR_DIESEL_SD
CAR_ETOH_E10_ELD
CAR_ETOH_E10_LD
CAR_ETOH_E10_MD
CAR_ETOH_E10_SD
CAR_ETOH_E85_ELD
CAR_ETOH_E85_LD
CAR_ETOH_E85_MD
CAR_ETOH_E85_SD
CAR_EV_ELD
CAR_EV_LD
CAR_EV_MD
CAR_EV_SD
CAR_FC_CH4_ELD
CAR_FC_CH4_LD
CAR_FC_CH4_MD
CAR_FC_CH4_SD
CAR_FC_H2_ELD
CAR_FC_H2_LD
CAR_FC_H2_MD
CAR_FC_H2_SD
CAR_GASOLINE_ELD
CAR_GASOLINE_LD
CAR_GASOLINE_MD
CAR_GASOLI

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 06/17/2024 13:53:51
  Finished: 06/17/2024 13:53:54
  Total time elapsed: 00:00:02
  CPU %: 51.60
  Memory %: 9.57


### Verification

In [7]:
new_db = load_extract_db(new_db_name, create_pickle=False)

Getting activity data


100%|██████████| 1543/1543 [00:00<00:00, 205265.35it/s]


Adding exchange data to activities


100%|██████████| 52318/52318 [00:02<00:00, 20652.34it/s]


Filling out exchange data


100%|██████████| 1543/1543 [00:01<00:00, 814.70it/s] 


In [14]:
res = []
for act in new_db:
    if 'Subject to double-counting removal (market)' in act['comment']: # we only look at operation LCI datasets that performed double-counting removal
        for exc in get_technosphere_flows(act):
            if ('maintenance' in exc['name']) | ('treatment' in exc['name']): # we keep maintenance and treatment in the operation LCI (proportional to use)
                pass
            elif ('EV charger' in exc['name']) | ('airport' in exc['name']): # these entries are proportional to the use phase also
                pass
            else:
                if (exc['amount'] != 0 ) & (exc['unit'] == 'unit'):
                    print(f"Activity: {act['name']}")
                    print(f"Exchange: {exc['name']}")
                    act_res = [a for a in db if a['code'] == exc['code']][0]
                    res.append(act_res['reference product'])
                else:
                    pass
    else:
        pass

Activity: tap water production, ultrafiltration treatment
Exchange: market for ultrafiltration module
Activity: tap water production, conventional treatment
Exchange: market for ultraviolet lamp
Activity: tap water production, underground water without treatment
Exchange: market for pump station
Activity: tap water production, underground water with chemical treatment
Exchange: market for pump station
Activity: diesel production, synthetic, Fischer Tropsch process, hydrogen from wood gasification, energy allocation
Exchange: gas-to-liquid plant construction
Activity: tap water production, underground water with disinfection
Exchange: market for pump station


In [15]:
list(set(res))

['ultraviolet lamp',
 'gas-to-liquid plant',
 'pump station',
 'ultrafiltration module']

## New database with foreground regionalization

### Database creation

In [6]:
my_ranking = [
    'CA-QC',
    'CA',
    'CA-ON',
    'CA-AB',
    'CA-BC',
    'CA-MB',
    'CA-NB',
    'CA-NF',
    'CA-NS',
    'CA-NT',
    'CA-NU',
    'CA-PE',
    'CAZ',
    'RNA',
    'US',
    'USA',
    'GLO',
    'RoW',
    'RER',
]

In [7]:
mismatch_regions = ['GLO', 'RoW', 'RER', 'CH'] # insufficient match within ecoinvent

In [8]:
create_esm_database(
    mapping=mapping_technologies, 
    model=model, 
    tech_specifics=technology_specifics, 
    technology_compositions=technology_compositions, 
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC, 
    main_database=db,
    esm_db_name=new_db_name+"_reg",
    results_path_file='energyscope_data/results/',
    regionalize_foregrounds=True,
    mismatch_regions=mismatch_regions,
    target_region='CA-QC',
    locations_ranking=my_ranking
)

AEC_OG
ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_BIOMETHANE
BIOGAS_SMR
BIOGAS_SMR_CCS
BIOMASS_ETHANOL
BIOMASS_GAS_EF_H2
BIOMASS_GAS_EF_H2_CCS
BIOMASS_GAS_FB_H2
BIOMASS_GAS_FB_H2_CCS
BUS_BIODIESEL_B100_SD
BUS_BIODIESEL_B20_SD
BUS_CNG_SD
BUS_DIESEL_SD
BUS_EV_SD
BUS_FC_CH4_SD
BUS_FC_H2_SD
BUS_HY_DIESEL_SD
BUS_PROPANE_SD
CAR_BIODIESEL_B100_ELD
CAR_BIODIESEL_B100_LD
CAR_BIODIESEL_B100_MD
CAR_BIODIESEL_B100_SD
CAR_BIODIESEL_B20_ELD
CAR_BIODIESEL_B20_LD
CAR_BIODIESEL_B20_MD
CAR_BIODIESEL_B20_SD
CAR_CNG_ELD
CAR_CNG_LD
CAR_CNG_MD
CAR_CNG_SD
CAR_DIESEL_ELD
CAR_DIESEL_LD
CAR_DIESEL_MD
CAR_DIESEL_SD
CAR_ETOH_E10_ELD
CAR_ETOH_E10_LD
CAR_ETOH_E10_MD
CAR_ETOH_E10_SD
CAR_ETOH_E85_ELD
CAR_ETOH_E85_LD
CAR_ETOH_E85_MD
CAR_ETOH_E85_SD
CAR_EV_ELD
CAR_EV_LD
CAR_EV_MD
CAR_EV_SD
CAR_FC_CH4_ELD
CAR_FC_CH4_LD
CAR_FC_CH4_MD
CAR_FC_CH4_SD
CAR_FC_H2_ELD
CAR_FC_H2_LD
CAR_FC_H2_MD
CAR_FC_H2_SD
CAR_GASOLINE_ELD
CAR_GASOLINE_LD
CAR_GASOLINE_MD
CAR_GASOLI

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 06/17/2024 14:29:39
  Finished: 06/17/2024 14:29:42
  Total time elapsed: 00:00:02
  CPU %: 41.90
  Memory %: 9.59
